# How we do it in Barbera1 - SIGNAL

We use Terrier 4.2. with Signal 1 M dataset

The script we used to convert JSON to TERRIER files did not include TITLE, so we indexed just the BODY.
We will use the TITLE (unprocessed) as query.

Index properties:
- we have only BODY
- we have a positional index: block.indexing=True


We apply BM25Classic and BM25Passage.
- for BM25Passage we divide the file in 10 equal parts
- for BM25P we apply the weights from out statistical analysis of the text on (TFIDF and IDF) - see notebook High IDF Terms in Signal collection.ipynb
- we evaluate: P@1, NDCG@k, Recall@k

## 1. Creating queries

In [9]:
%load_ext autoreload
%autoreload 2

%store -r dataset

from elasticsearch import Elasticsearch, RequestError
import json

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [10]:
dataset

,content,id,media-type,published,source,title,all_text
0,VETERANS saluted Worcester's first ever breakf...,f7ca322d-c3e8-40d2-841f-9d7250ac72ca,News,2015-09-07T10:16:14Z,Redditch Advertiser,Worcester breakfast club for veterans gives hu...,Worcester breakfast club for veterans gives hu...
1,New Product Gives Marketers Access to Real Key...,609772bc-0672-4db5-8516-4c025cfd54ca,News,2015-09-17T15:00:00Z,Virtualization Conference & Expo,Jumpshot Gives Marketers Renewed Visibility In...,Jumpshot Gives Marketers Renewed Visibility In...
2,Home »\rStyle » The Return Of The Nike Air Max...,1aa9d1b0-e6ba-4a48-ad0c-66552d896aac,Blog,2015-09-22T22:54:37Z,Streets Connect,The Return Of The Nike Air Max Sensation Has 8...,The Return Of The Nike Air Max Sensation Has 8...
3,NYMag.com Daily Intelligencer Vulture The Cut ...,719699f9-47be-4bc7-969b-b53a881c95ae,Blog,2015-09-16T23:12:11Z,The Cut,This New Dating App Will Ruin Your Internet Game,This New Dating App Will Ruin Your Internet Ga...
4,"KUALA LUMPUR, Sept 15 (MySinchew) -- The Kuala...",a080f99a-07d9-47d1-8244-26a540017b7a,News,2015-09-15T10:17:53Z,My Sinchew,Pay up or face legal action: DBKL,Pay up or face legal action: DBKL \n KUALA LUM...
5,"Mumbai, Sep 11 : Following were the indicative...",3c5636e8-06b7-43fd-8a80-0e8535f45cb2,News,2015-09-11T06:09:32Z,Newshence.com,"Euro up; USD, Pound and Yen down","Euro up; USD, Pound and Yen down \n Mumbai, Se..."
6,Words kill or give life; they are usually eith...,ba44d0cd-bff2-4875-8036-86f37419b5e7,News,2015-09-28T23:21:00Z,Times of Swaziland,THE INFLUENCE OF OUR WORDS,THE INFLUENCE OF OUR WORDS \n Words kill or gi...
7,"Just in time for the holiday shopping seasons,...",402f6983-3300-4546-82c5-6f4eb08d0ba7,News,2015-09-10T03:25:04Z,South Coast Register,"Hands on with Apple's iPhone 6s, iPad Pro and ...","Hands on with Apple's iPhone 6s, iPad Pro and ..."
8,SOURCE Harwood Feffer LLP\n\nNEW YORK \n\nOn J...,286868ed-ab9a-4990-a4af-436df8c5bff2,News,2015-09-25T21:00:00Z,WAFB,Harwood Feffer LLP Announces Investigation of ...,Harwood Feffer LLP Announces Investigation of ...
9,Tony Abbott's last word \nWatch the last pres...,c5f8a528-cc0f-4f3e-aaef-b9e3b6b00325,News,2015-09-15T06:04:06Z,Brisbane Times,What will happen to Peta Credlin and Brian Lou...,What will happen to Peta Credlin and Brian Lou...


In [ ]:
### make file from title
# dataset.title.to_csv('signal-title-queries.txt', sep=':') #, index=False)

In [ ]:
newTitle = dataset.title.str.replace('[^a-zA-Z ]', '')

In [ ]:
newTitle[0]

In [ ]:
# newTitle.to_csv('signal-title-queries.txt', sep=':')

## 2.Running the queries on Terrier (outside job)

#### BM25Classic

My command - not using terrier passage:

    bin/trec_terrier.sh -r -Dtrec.model=BM25  
                           -Dtrec.topics=/home/muntean/signal-news/signal-title-queries.txt 
                           -Dtrec.results=/home/muntean/signal-news/ 
                           -Dtrec.results.file=BM25-signal.res;
                           
05:45:21.757 [main] INFO  o.t.a.batchquerying.TRECQuerying - Settings of Terrier written to /home/muntean/signal-news/BM25-signal.res.settings

05:45:21.757 [main] INFO  o.t.a.batchquerying.TRECQuerying - Finished topics, executed 999306 queries in 148734.905 seconds, results written to /home/muntean/signal-news/BM25-signal.res

Time elapsed: 148739.591 seconds.   aprox. 41 hours 
    


#### BM25Passage 

This is equivalent to classic: 
    bin/trec_terrier.sh -r -Dtrec.model=models.BM25P10 
                           -Dbm25p.w=[1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0] 
                           -Dtrec.topics=/home/muntean/signal-news/signal-title-queries.txt 
                           -Dtrec.results=/home/muntean/signal-news/
                           -Dtrec.results.file=BM25P10-signal-all-ones.res 
 __skipped due to huge output files__
                           
This takes the distribution from http://146.48.82.32:9999/notebooks/signal-news/High%20IDF%20Terms%20in%20Signal%20collection.ipynb#
 
##### idf (top 5): array([ 0.21536741,  0.07443482,  0.0717614 ,  0.06874424,  0.06757452, 0.0709799 ,  0.06936171,  0.06951786,  0.07363883,  0.21861932])

    bin/trec_terrier.sh -r -Dtrec.model=models.BM25P10 
                           -Dbm25p.w=[0.21536741,0.07443482,0.0717614,0.06874424,0.06757452,0.0709799,0.06936171,0.06951786,0.07363883,0.21861932] 
                           -Dtrec.topics=/home/muntean/signal-news/signal-title-queries.txt 
                           -Dtrec.results=/home/muntean/signal-news/ 
                           -Dtrec.results.file=BM25P10-signal-idf.res
                           
21:50:15.621 [main] INFO  o.t.a.batchquerying.TRECQuerying - Settings of Terrier written to /home/muntean/signal-news/BM25P10-signal-idf.res.settings
21:50:15.621 [main] INFO  o.t.a.batchquerying.TRECQuerying - Finished topics, executed 999306 queries in 189875.684 seconds, results written to /home/muntean/signal-news/BM25P10-signal-idf.res
Time elapsed: 189880.827 seconds.

##### idf (top 10): array([ 0.15849338,  0.08654554,  0.08584007,  0.08343284,  0.08223365, 0.08589565,  0.08470705,  0.08382871,  0.08707116,  0.16195194])

    bin/trec_terrier.sh -r -Dtrec.model=models.BM25P10 
                           -Dbm25p.w=[0.15849338,0.08654554,0.08584007,0.08343284,0.08223365,0.08589565,0.08470705,0.08382871,0.08707116,0.16195194] 
                           -Dtrec.topics=/home/muntean/signal-news/signal-title-queries.txt 
                           -Dtrec.results=/home/muntean/signal-news/ 
                           -Dtrec.results.file=BM25P10-signal-idf-top10.res
                           
21:18:49.976 [main] INFO  o.t.a.batchquerying.TRECQuerying - Settings of Terrier written to /home/muntean/signal-news/BM25P10-signal-idf-top10.res.settings
21:18:49.977 [main] INFO  o.t.a.batchquerying.TRECQuerying - Finished topics, executed 999306 queries in 186873.272 seconds, results written to /home/muntean/signal-news/BM25P10-signal-idf-top10.res
Time elapsed: 186880.933 seconds.

###### tfidf (top 5): array([ 0.16176254,  0.09472742,  0.09135097,  0.08879455,  0.0870066 , 0.09013596,  0.0879273 ,  0.08500273,  0.08299202,  0.13029989])
    
    bin/trec_terrier.sh -r -Dtrec.model=models.BM25P10 
                           -Dbm25p.w=[0.16176254,0.09472742,0.09135097,0.08879455,0.0870066,0.09013596,0.0879273,0.08500273,0.08299202,0.13029989] 
                           -Dtrec.topics=/home/muntean/signal-news/signal-title-queries.txt 
                           -Dtrec.results=/home/muntean/signal-news/ 
                           -Dtrec.results.file=BM25P10-signal-tfidf.res
                           
22:27:27.727 [main] INFO  o.t.a.batchquerying.TRECQuerying - Settings of Terrier written to /home/muntean/signal-news/BM25P10-signal-tfidf.res.settings
22:27:27.727 [main] INFO  o.t.a.batchquerying.TRECQuerying - Finished topics, executed 999306 queries in 191999.467 seconds, results written to /home/muntean/signal-news/BM25P10-signal-tfidf.res
Time elapsed: 192004.228 seconds.

###### tfidf (top 10): array([ 0.14250787,  0.0989011 ,  0.09575329,  0.09325474,  0.09129108,  0.09466752,  0.09278141,  0.08985508,  0.08810683,  0.11288108])

    bin/trec_terrier.sh -r -Dtrec.model=models.BM25P10 
                           -Dbm25p.w=[0.14250787,0.0989011,0.09575329,0.09325474,0.09129108,0.09466752,0.09278141,0.08985508,0.08810683,0.11288108] 
                           -Dtrec.topics=/home/muntean/signal-news/signal-title-queries.txt 
                           -Dtrec.results=/home/muntean/signal-news/ 
                           -Dtrec.results.file=BM25P10-signal-tfidf-top10.res

22:29:36.826 [main] INFO  o.t.a.batchquerying.TRECQuerying - Settings of Terrier written to /home/muntean/signal-news/BM25P10-signal-tfidf-top10.res.settings
22:29:36.826 [main] INFO  o.t.a.batchquerying.TRECQuerying - Finished topics, executed 999306 queries in 190462.093 seconds, results written to /home/muntean/signal-news/BM25P10-signal-tfidf-top10.res
Time elapsed: 190469.4 seconds.

## 3. Running the queries on Elastic!
#### JUST for BM25 so as to find query types
An article has the following fields:  

```{'_index': 'articles', 
    '_type': 'article', 
    '_id': '26', 
    '_score': 1.0, 
    '_source': {'title': "Survivor Upset Use of 'Eye of the Tiger' With Kim Davis", 
                'media-type': 'Blog', 
                'content': 'On Tuesday afternoon, Kim Davis, the Kentucky clerk whose imprisonment for not granting same-sex marriage licenses became a cause célèbre for the right, was released amid fanfare and a glowing introduction by Republican presidential hopeful Mike Huckabee.As Davis, her husband and attorney ...', 
                'source': 'Latest News on One News Page [United States] - Top Headlines and News Videos', 
                'published': '2015-09-10T03:00:15Z', 
                'id': '7516303b-0db5-477d-9e5d-243a73865e39'}}```

In [2]:
es = Elasticsearch(['http://localhost/'], 
                    #http_auth=('elastic', 'bm25p'),
                    port=9200,
                    timeout=30
                    )

In [6]:
res = es.search(index="articles", 
                doc_type='article', 
                size=10, 
                body={"query": {
                        "match": {
                                "content": "Dillard"
                        }}})

# [x['_source'] for x in res['hits']['hits']]
for r in res['hits']['hits'][:3]:
    print(r)
    print()

{'_index': 'articles', '_type': 'article', '_id': '858348', '_score': 16.706663, '_source': {'title': 'White demands answers about stolen EMS generator', 'media-type': 'News', 'content': 'Precinct 1 Commissioner Danny White Monday demanded answers about a stolen generator that’s been missing from Wise County Emergency Medical Services for more than six months. \n \n“I’m not happy about this,” he said at the commissioners meeting. “It’s a $1,000 generator, and I don’t think we should just say, ‘it’s stolen.’ If it was any other place else, I think we would be investigating it. \n \n“I don’t feel like, as a county commissioner, we need to sweep it under the rug,” White said. “If it calls for a lie detector test, bloodwork or whatever, let’s do it.” \n \nEMS Administrator Charles Dillard sat speechless. \n \nSheriff David Walker said he could check with his investigators, but he didn’t know of any new leads. \n \n“As far as the criminal stuff, we’ve taken it as far as we can go,” Walker s